In [0]:
import numpy as np
import tensorflow as tf
import os

In [2]:
if tf.gfile.Exists('./fenwicks'):
  tf.gfile.DeleteRecursively('./fenwicks')
!git clone https://github.com/fenwickslab/fenwicks.git

import fenwicks as fw

Cloning into 'fenwicks'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 479 (delta 36), reused 44 (delta 18), pack-reused 413
Receiving objects: 100% (479/479), 185.02 KiB | 3.78 MiB/s, done.
Resolving deltas: 100% (252/252), done.


In [0]:
BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
EPOCHS = 24 #@param {type:"integer"}
WARMUP = 5 #@param {type:"integer"}
BUCKET = 'gs://gs_colab' #@param {type:"string"}
PROJECT = 'cifar10' #@param {type:"string"}

In [4]:
fw.colab_tpu.setup_gcs()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
data_dir, work_dir = fw.io.get_gcs_dirs(BUCKET, PROJECT)

In [0]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
n_train, n_test = X_train.shape[0], X_test.shape[0]
img_size = X_train.shape[1]
n_classes = y_train.max() + 1

In [0]:
X_train_mean = np.mean(X_train, axis=(0,1,2))
X_train_std = np.std(X_train, axis=(0,1,2))
X_train = (X_train - X_train_mean) / X_train_std
X_test = (X_test - X_train_mean) / X_train_std

In [0]:
train_fn = os.path.join(data_dir, "train.tfrec")
test_fn = os.path.join(data_dir, "test.tfrec")

fw.io.numpy_tfrecord(X_train, y_train, train_fn)
fw.io.numpy_tfrecord(X_test, y_test, test_fn)

In [0]:
def train_parser(tfexample):
  x, y = fw.io.tfexample_numpy_image_parser(tfexample, img_size, img_size)
  x = fw.transform.ramdom_pad_crop(x, 4)
  x = fw.transform.random_flip(x)
  x = fw.transform.cutout(x, 8, 8)
  return x, y

test_parser = lambda x: fw.io.tfexample_numpy_image_parser(x, img_size, img_size)

In [0]:
train_input_func = lambda params: fw.io.tfrecord_ds(train_fn, train_parser, batch_size=params['batch_size'], training=True)
eval_input_func = lambda params: fw.io.tfrecord_ds(test_fn, test_parser, batch_size=params['batch_size'], training=False)

In [0]:
def build_nn(c=64, weight=0.125):
  model = fw.Sequential()
  model.add(fw.layers.ConvBN(c, **fw.layers.PYTORCH_CONV_PARAMS))
  model.add(fw.layers.ConvResBlk(c*2, res_convs=2, **fw.layers.PYTORCH_CONV_PARAMS))
  model.add(fw.layers.ConvBlk(c*4, **fw.layers.PYTORCH_CONV_PARAMS))
  model.add(fw.layers.ConvResBlk(c*8, res_convs=2, **fw.layers.PYTORCH_CONV_PARAMS))
  model.add(tf.keras.layers.GlobalMaxPool2D())
  model.add(tf.keras.layers.Dense(n_classes, kernel_initializer=fw.layers.init_pytorch, use_bias=False))
  model.add(fw.layers.Scaling(weight))
  return model

In [0]:
steps_per_epoch = n_train // BATCH_SIZE

In [0]:
opt_func = fw.train.sgd_triangle_one_cycle(EPOCHS*steps_per_epoch, lr=LEARNING_RATE/BATCH_SIZE, warmup_steps=WARMUP*steps_per_epoch, mom=MOMENTUM)
model_func = fw.tpuest.get_clf_model_func(build_nn, opt_func, reduction=tf.losses.Reduction.SUM)

In [14]:
est = fw.tpuest.get_tpu_estimator(n_train, n_test, model_func, work_dir, trn_bs=BATCH_SIZE)
est.train(train_input_func, steps=steps_per_epoch*EPOCHS)

INFO:tensorflow:Using config: {'_model_dir': 'gs://gs_colab/work/cifar10-2019-04-10-17:46:37', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.84.91.18:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f644e829b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.84.91.18:8470', '_evaluation_master': 'grpc://10.84.91.18:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=97, num_shards=None, num

In [15]:
result = est.evaluate(eval_input_func, steps=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-10T17:47:42Z
INFO:tensorflow:TPU job name worker
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from gs://gs_colab/work/cifar10-2019-04-10-17:46:37/model.ckpt-2328
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Init TPU system
INFO:tensorflow:Initialized TPU in 10 seconds
INFO:tensorflow:Starting infeed thread controller.
INFO:tensorflow:Starting outfeed thread controller.
INFO:tensorflow:Initialized dataset iterators in 0 seconds
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Stop infeed thread controller
INFO:tensorflow:Shutting down InfeedController thread.
INFO:tensorflow:In

In [16]:
print(f'Test results: accuracy={result["accuracy"] * 100: .2f}%, loss={result["loss"]: .2f}.')

Test results: accuracy= 93.28%, loss= 227.06.


In [0]:
fw.io.create_clean_dir(work_dir)